In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import numpy as np
import pandas as pd

In [4]:
## load the trained model,scaler pickle ,onehot
model=load_model('model.h5')

## load the encoder and scaler
with open('onehot_encoder_geo.pkl','rb') as file:
    label_encoder_geo=pickle.load(file)
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)
with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)    

In [6]:
## example input data
input_data = {
    'CreditScore': '600',
    'Geography': 'France',
    'Gender': 'Male',
    'Age': '40',
    'Tenure': '3',
    'Balance': '60000',
    'NumOfProducts': '2',
    'HasCrCard': '1',
    'IsActiveMember': '1',
    'EstimatedSalary': 50000
}


In [12]:
# One Hot Encode 'Geography'
geo_encoded=label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df



c:\Users\kesha\OneDrive\Desktop\ANN CLASSIFICATION\venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [13]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [14]:
# Encode Categorical variables
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [22]:
print(input_df.columns)

Index(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Geography_France',
       'Geography_Germany', 'Geography_Spain'],
      dtype='object')


In [23]:
input_df = pd.concat(
    [input_df.drop(columns=['Geography'], errors='ignore'), geo_encoded_df],
    axis=1
)


In [28]:
input_df = input_df.reindex(
    columns=scaler.feature_names_in_,
    fill_value=0
)

input_scaled = scaler.transform(input_df)
input_scaled



array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802,  1.00150113, -0.57946723, -0.57638802]])

In [29]:
## predict churn
prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 282ms/step


array([[0.02533334]], dtype=float32)

In [30]:
prediction_proba=prediction[0][0]
prediction_proba

0.025333337

In [32]:
if prediction_proba > 0.5:
    print("The customer is likely to churn.")   
else:
    print("The customer is unlikely to churn.")
    

The customer is unlikely to churn.
